In [28]:
import pandas as pd
import os
from pathlib import Path
from dotenv import load_dotenv
import spotipy
from tqdm import tqdm
import numpy as np
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [39]:
request_timer = 0.33
person = "selina"
data_source = "./data/" + person

load_dotenv()
ID = os.getenv("ID_B")
SECRET = os.getenv("SECRET_B")

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=ID, client_secret=SECRET))

In [33]:
paths = Path(data_source).glob("Streaming*.json")
df = pd.concat(map(pd.read_json, paths))
df = df[df["spotify_track_uri"].notnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 233983 entries, 0 to 4730
Data columns (total 21 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   ts                                 233983 non-null  object 
 1   username                           233983 non-null  object 
 2   platform                           233983 non-null  object 
 3   ms_played                          233983 non-null  int64  
 4   conn_country                       233983 non-null  object 
 5   ip_addr_decrypted                  229740 non-null  object 
 6   user_agent_decrypted               229312 non-null  object 
 7   master_metadata_track_name         233983 non-null  object 
 8   master_metadata_album_artist_name  233983 non-null  object 
 9   master_metadata_album_album_name   233983 non-null  object 
 10  spotify_track_uri                  233983 non-null  object 
 11  episode_name                       0 non-null 

# 1. Preparation

## 1.1 Unique Tracks with every "spotify_track_uri"

In [34]:
tracks_album = df[df["episode_name"].isnull()][["master_metadata_track_name", "master_metadata_album_album_name", "spotify_track_uri", "master_metadata_album_artist_name"]].drop_duplicates()
tracks_album = tracks_album.dropna()
tracks_album.to_csv("./processed/" + person + "/tracks_album.csv", index=False)

Drop all duplicate entries in the history, therey removing multiple plays.
However this is not sufficient to properly eliminate duplicate references to the same track, since one track can be played from different albums.
For Example, when a song is at first released as a single, and later within an album. Individual plays appear with a different `spotify_track_uri`, altough they should be counted as one.

To fix this, the online tool [Datablist](https://datablist.com) is used, to find and remove duplicates.
An entry is considered a duplicate if the artist, album name, and track name are identical.
Subsequently, the track uri's are merged into a comma-separated string, which is then transformed into an array for futher processing.
Tough the column is then no longer atomic, is is now easier to assign play entries, with different uris to the same song.

1. Import `tracks_album.csv` into Datablist
2. Find duplicates based on `artist`, `track` and `album`
3. Merge `spotify_track_uri`, drop `album`
4. Save as `tracks_all_uris.csv`

<div class="alert alert-block alert-info">
<b>TODO:</b>    
This should be automated, for processing new data.
</div>

In [36]:
tracks_all_uris = pd.read_csv("./processed/" + person + "/datablist_export.csv")
tracks_all_uris["spotify_track_uri"] = tracks_all_uris["spotify_track_uri"].apply(lambda r: [s for s in r.split(",")])
print("Unique tracks after cleaning:", len(tracks_all_uris))
tracks_all_uris.to_csv("./processed/" + person + "/tracks_all_uris.csv", index=False)

Unique tracks after cleaning: 32644


The redundant uris are no longer needed and can now be excluded.

In [112]:
tracks_all_uris["spotify_track_uri"] = tracks_all_uris["spotify_track_uri"].str.replace("\[|\]|\'", "", regex=True).str.split(",").map(lambda r:r[0])
tracks_all_uris.to_csv("./processed/" + person + "/tracks_unique.csv", index=False)

## 1.2 History fix

This separation still is not enough, because having two keys for every track makes it difficult to aggregate the data. Therefore every uri in the extended history, that is related to another track, is replaced by the first uri in the spotify_track_uri array above.

In [177]:
process = False
if process:
    tracks_all_uris = pd.read_csv("./processed/" + person + "/tracks_all_uris.csv")
    lookup = tracks_all_uris["spotify_track_uri"].to_frame()
    lookup.columns = ["from"]
    lookup["to"] = lookup["from"].map(lambda r: r[0])
    lookup.head()
    
    tqdm.pandas()
    df["spotify_track_uri"] = df["spotify_track_uri"].progress_map(lambda x: lookup[lookup["from"].str.join(" ").str.contains(x)]["to"].item())
    df.drop(["Unnamed: 0", "username", "platform", "ip_addr_decrypted", "user_agent_decrypted", "master_metadata_album_artist_name", "master_metadata_album_album_name", "episode_name", "episode_show_name", "spotify_episode_uri", "offline", "offline_timestamp", "incognito_mode"], axis=1).to_csv("./processed/" + person + "/history_cleaned.csv", index=False)

## 1.3 Audio Features

Once the tracks have been processed, the next stage is to prepare the relevant metrics.
The Spotify web API provides information about every track, namely audio features and analysis.

This is an example response for the audio features of "Time is Running Out" by "Muse".
```json
"acousticness": 0.00242,
"analysis_url": "https://api.spotify.com/v1/audio-analysis/2takcwOaAZWiXQijPHIx7B",
"danceability": 0.585,
"duration_ms": 237040,
"energy": 0.842,
"id": "2takcwOaAZWiXQijPHIx7B",
"instrumentalness": 0.00686,
"key": 9,
"liveness": 0.0866,
"loudness": -5.883,
"mode": 0,
"speechiness": 0.0556,
"tempo": 118.211,
"time_signature": 4,
"track_href": "https://api.spotify.com/v1/tracks/2takcwOaAZWiXQijPHIx7B",
"type": "audio_features",
"uri": "spotify:track:2takcwOaAZWiXQijPHIx7B",
"valence": 0.428
```

Since the uri's are now cleaned, we can just request the auto features via the spotify api and append them to our dataframe.

In [38]:
audio_features = pd.DataFrame(columns=[
    "spotify_track_uri",
    "acousticness",
    "danceability",
    "duration_ms",
    "energy",
    "instrumentalness",
    "key",
    "liveness",
    "loudness",
    "mode",
    "speechiness",
    "tempo",
    "time_signature",
    "valence",
])

In [113]:
tracks_unique = pd.read_csv("./processed/" + person + "/tracks_unique.csv")

In [114]:
process = False
if process:
    for chunk in tqdm(np.array_split(tracks_unique, len(tracks_unique)/90)):
        try:
            time.sleep(request_timer)
            uris = chunk["spotify_track_uri"].to_numpy()
            result = sp.audio_features(uris)
            result_nonull = []
            for obj in result:
                if obj != None:
                    result_nonull.append(obj)
            chunk_df = pd.DataFrame(result_nonull)
            audio_features = pd.concat([audio_features, chunk_df], join="inner")
        except Exception as e:
            print(e)
            break
    audio_features.to_csv("./processed/" + person + "/audio_features.csv", index=False)
    missed = len(tracks_all_uris) - len(audio_features)
    print(missed, "tracks missing")
    tracks_unique[~tracks_unique["spotify_track_uri"].isin(audio_features["uri"])]

The subsequent stage is to merge the audio feature into the dataframe with the unique tracks.

In [123]:
tracks_features = tracks_unique.merge(audio_features, how="outer", on="spotify_track_uri")
tracks_features[tracks_features["energy"].notnull()]
tracks_features.to_csv("./processed/" + person + "/tracks_features.csv", index=False)
tracks_features.head(3)

,master_metadata_track_name,master_metadata_album_album_name,spotify_track_uri,master_metadata_album_artist_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Still Got Time (feat. PARTYNEXTDOOR),Still Got Time (feat. PARTYNEXTDOOR),spotify:track:000xQL6tZNLJzIrtIgxqSl,ZAYN,0.13100,0.748,188491,0.627,0.000,7,0.0852,-6.029,1,0.0639,120.963,4,0.524
1,Be Alright (feat. EMEL),Be Alright (feat. EMEL),spotify:track:003U4oQXv82GYo1tNquN2F,Stonebank,0.00216,0.538,299294,0.770,0.165,7,0.3550,-3.924,0,0.1030,170.035,4,0.235
2,Mr. Brightside,Hot Fuss,spotify:track:003vvx7Niy0yvhvHt4a68B,The Killers,0.00121,0.352,222973,0.911,0.000,1,0.0995,-5.230,1,0.0747,148.033,4,0.236


# 2. Artists and Genres

In order to gain insight into the different genres listened to, the artist data must be queried. Due to the nature of an n to n relationship, the artist and genre data is modelled as shown in the ER diagram above.

The problem is that only the main artist is given in the extended history, not the features. Also, the artist is only referenced by name and a spotify_id is required to request further data. However, the tracks api also contains all information about the artist that contributed to the track. So in order to get the artist ids, we have to querry all tracks.

## 2.1 Artists

In [124]:
tracks_unique = pd.read_csv("./processed/" + person + "/tracks_unique.csv")
tracks_unique.head(3)

,master_metadata_track_name,master_metadata_album_album_name,spotify_track_uri,master_metadata_album_artist_name
0,Womanizer,Circus (Deluxe Version),spotify:track:4fixebDZAVToLbUCuEloa2,Britney Spears
1,Boom Boom Pow,THE E.N.D. (THE ENERGY NEVER DIES),spotify:track:3oDFtOhcN08qeDPAK6MEQG,Black Eyed Peas
2,Shut Up And Drive,Good Girl Gone Bad: Reloaded,spotify:track:4Tn2llBm1g0UlWctmgPL8Z,Rihanna


In [157]:
process = True
if process:
    artist_data = []
    artist_track_data = []
    for chunk in tqdm(np.array_split(tracks_unique["spotify_track_uri"], len(tracks_unique)/45)):
        try:
            uris = chunk.to_numpy()
            result = sp.tracks(uris, market="DE")["tracks"]
            time.sleep(request_timer)
            result_nonull = []
            for track in result:
                if track != None:
                    for artist in track["artists"]:
                        artist_data.append([artist["name"], artist["uri"], artist.get("genres", None)])
                        artist_track_data.append([artist["uri"], track["uri"]])
                    
        except Exception as e:
            print(e)
            break

    artists_tracks = pd.DataFrame(artist_track_data, columns=["artist_uri", "track_uri"])
    artists_tracks.to_csv("./processed/" + person + "/artist_track.csv", index=False)
    
    artists = pd.DataFrame(artist_data, columns=["artist_name", "artist_uri", "genres"]).drop_duplicates()
    artists.to_csv("./processed/" + person + "/artists.csv", index=False)

/home/voss/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████████████████████████████████████████████████| 725/725 [06:37<00:00,  1.83it/s]


## 2.2 Genres

In [158]:
artists = pd.read_csv("./processed/" + person + "/artists.csv")
artists.head(3)

,artist_name,artist_uri,genres
0,Britney Spears,spotify:artist:26dSoYclwsYLMAKD3tpOr4,NaN
1,Black Eyed Peas,spotify:artist:1yxSLGMDHlW21z4YXirZDS,NaN
2,Rihanna,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,NaN


In [162]:
request = True
if request:
    artist_genres = []
    for chunk in tqdm(np.array_split(artists["artist_uri"], len(artists)/45)):
        try:
            uris = chunk.to_numpy()
            result = sp.artists(uris)["artists"]
            time.sleep(request_timer)
            for artist in result:
                artist_genres.append([artist["uri"], artist["genres"]])
                
        except Exception as e:
            print(e)
            break

100%|██████████████████████████████████████████████████████| 409/409 [03:38<00:00,  1.87it/s]


In [166]:
artist_genres_arrays = pd.DataFrame(artist_genres, columns=["artist_uri", "genres"])
artist_genres_arrays.head(3)

,artist_uri,genres
0,spotify:artist:26dSoYclwsYLMAKD3tpOr4,"[dance pop, pop]"
1,spotify:artist:1yxSLGMDHlW21z4YXirZDS,"[dance pop, pop, pop rap]"
2,spotify:artist:5pKCCKE2ajJHZ9KAiaK11H,"[barbadian pop, pop, urban contemporary]"


In [176]:
genres = pd.DataFrame(sum(artist_genres_arrays["genres"].tolist(), []), columns=["genre"]).drop_duplicates()
genres.head(3)

genre_artist_data = []
for i, artist in artist_genres_arrays.iterrows():
    artist_genres = artist["genres"]
    for genre_id in genres[genres["genre"].isin(artist_genres)].to_numpy():
        genre_artist_data.append([artist["artist_uri"], genre_id[0]])

artist_to_genre = pd.DataFrame(genre_artist_data, columns=["artist_uri", "genre_id"])
artist_to_genre.to_csv("./processed/" + person + "/artist_genre.csv")